## Imports

In [1]:
# !pip install selenium 
# !pip install webdriver_manager
# !pip install validators

  Using cached validators-0.19.0-py3-none-any.whl


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import requests 
from requests_html import HTMLSession, AsyncHTMLSession
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import urllib.request, urllib.parse
import os
import validators

In [ ]:
'''browser_options = Options()
browser_options.headless = True
browser_options.add_argument('disable-blink-features=AutomationControlled')
browser_options.add_argument('--lang=en_US') '''

# https://stackoverflow.com/questions/60415144/python-selenium-headless-chromedriver-not-loading-full-page-when-it-was-working

In [65]:
service = Service(executable_path=ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\nopto\.wdm\drivers\chromedriver\win32\101.0.4951.41]


## Functions

In [3]:
# https://www.geeksforgeeks.org/image-scraping-with-python/
# https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python
# https://www.analyticsvidhya.com/blog/2020/08/web-scraping-selenium-with-python/

def getdata(url, driver):
    driver.get(url)
    driver.implicitly_wait(10)
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
 
    return driver.page_source

def getimages(name, driver):
    imagelist = []
    link = "https://pokemondb.net/sprites/" + name
    htmldata = getdata(link, driver) 
    soup = BeautifulSoup(htmldata, 'html.parser') 
    for item in soup.find_all('img'):
        if item.has_attr('src'):
            imagelist.append(item['src'])
    
    return imagelist

## Create List of names to scrape

In [4]:
poke_df = pd.read_csv("Data/pokemonstats.csv")

In [5]:
poke_df.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

In [6]:
poke_names = poke_df[['name', 'pokedex_number', 'type1']]

### Change name formatting to match website

In [83]:
poke_names.loc[771]

name              Type: Null
pokedex_number           772
type1                 normal
Name: 771, dtype: object

In [61]:
poke_names = poke_names.replace('Nidoran♀', 'nidoran-f')
poke_names = poke_names.replace('Nidoran♂', 'nidoran-m')
poke_names = poke_names.replace("Farfetch'd",'farfetchd')
poke_names = poke_names.replace('Mr. Mime','mr-mime')
poke_names = poke_names.replace('Mime Jr.','mime-jr')
poke_names = poke_names.replace('Flabébé','flabebe')
poke_names = poke_names.replace('Type: Null','type-null')
poke_names = poke_names.replace('Tapu Koko','tapu-koko')
poke_names = poke_names.replace('Tapu Lele','tapu-lele')
poke_names = poke_names.replace('Tapu Bulu','tapu-bulu')
poke_names = poke_names.replace('Tapu Fini','tapu-fini')

In [62]:
poke_name_list = poke_names['name'].to_list()

## Testing

In [82]:
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(15)

In [12]:
snorraw = getdata("https://pokemondb.net/sprites/Snorlax", driver)

In [14]:
snorsoup = BeautifulSoup(snorraw, 'html.parser') 

In [16]:
snorsoup.find_all("img")

[<img alt="Snorlax  sprite from Red &amp; Blue" class="img-fixed img-sprite-v1" loading="lazy" src="https://img.pokemondb.net/sprites/red-blue/normal/snorlax.png"/>,
 <img alt="Snorlax  sprite from Silver" class="img-fixed img-sprite-v3" loading="lazy" src="https://img.pokemondb.net/sprites/silver/normal/snorlax.png"/>,
 <img alt="Snorlax  sprite from Ruby &amp; Sapphire" class="img-fixed img-sprite-v5" loading="lazy" src="https://img.pokemondb.net/sprites/ruby-sapphire/normal/snorlax.png"/>,
 <img alt="Snorlax  sprite from Diamond &amp; Pearl" class="img-fixed img-sprite-v8" loading="lazy" src="https://img.pokemondb.net/sprites/diamond-pearl/normal/snorlax.png"/>,
 <img alt="Snorlax  sprite from Black &amp; White" class="img-fixed img-sprite-v11" loading="lazy" src="https://img.pokemondb.net/sprites/black-white/normal/snorlax.png"/>,
 <img alt="Snorlax  sprite from X &amp; Y" class="img-fixed img-sprite-v13" loading="lazy" src="https://img.pokemondb.net/sprites/x-y/normal/snorlax.png"

In [17]:
snorlist = []
for item in snorsoup.find_all('img'):
        snorlist.append(item['src'])

In [18]:
snorlist

['https://img.pokemondb.net/sprites/red-blue/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/silver/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/ruby-sapphire/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/diamond-pearl/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/black-white/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/x-y/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/sun-moon/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/sword-shield/normal/snorlax.png',
 'https://img.pokemondb.net/sprites/silver/shiny/snorlax.png',
 'https://img.pokemondb.net/sprites/ruby-sapphire/shiny/snorlax.png',
 'https://img.pokemondb.net/sprites/diamond-pearl/shiny/snorlax.png',
 'https://img.pokemondb.net/sprites/black-white/shiny/snorlax.png',
 'https://img.pokemondb.net/sprites/x-y/shiny/snorlax.png',
 'https://img.pokemondb.net/sprites/sun-moon/shiny/snorlax.png',
 'https://img.pokemondb.net/sprites/home/normal/snorlax.png',
 'ht

In [83]:
snorlax = getimages('Snorlax', driver)

In [84]:
snorlax

['https://img.pokemondb.net/sprites/red-blue/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/silver/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/ruby-sapphire/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/diamond-pearl/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/black-white/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/x-y/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/lets-go-pikachu-eevee/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/sword-shield/normal/wartortle.png',
 'https://img.pokemondb.net/sprites/silver/shiny/wartortle.png',
 'https://img.pokemondb.net/sprites/ruby-sapphire/shiny/wartortle.png',
 'https://img.pokemondb.net/sprites/diamond-pearl/shiny/wartortle.png',
 'https://img.pokemondb.net/sprites/black-white/shiny/wartortle.png',
 'https://img.pokemondb.net/sprites/x-y/shiny/wartortle.png',
 'https://img.pokemondb.net/sprites/home/normal/wartortle.png',
 'https://img.pokemondb.net/

In [86]:
snor_df = pd.DataFrame()

In [87]:
snor_df.append(snorlax).T

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/ruby-sapphir...,https://img.pokemondb.net/sprites/diamond-pear...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/x-y/normal/w...,https://img.pokemondb.net/sprites/lets-go-pika...,https://img.pokemondb.net/sprites/sword-shield...,https://img.pokemondb.net/sprites/silver/shiny...,https://img.pokemondb.net/sprites/ruby-sapphir...,...,https://img.pokemondb.net/sprites/silver/back-...,https://img.pokemondb.net/sprites/yellow/norma...,https://img.pokemondb.net/sprites/yellow/norma...,https://img.pokemondb.net/sprites/yellow/back-...,https://img.pokemondb.net/sprites/yellow/back-...,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/red-blue/bac...,https://img.pokemondb.net/sprites/red-blue/bac...,https://ib.adnxs.com/getuid?https%3A%2F%2Fu.4d...


## For Loop to Scrape images for 801 pokemon in list

In [15]:
image_urls_df = pd.DataFrame()
test_list = []

driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.implicitly_wait(15)
counter = 0

for name in poke_name_list:
    urls = getimages(name, driver)
    urls = [name] + urls
    test_list.append(urls)
    temp_df = pd.DataFrame(urls).T
    image_urls_df = image_urls_df.append(temp_df)
    time.sleep(15)
    
    counter +=1
    
    # In order to avoid errors from too many requests in a short time: close driver, wait, open new driver 
    if (counter % 4) == 0:
        driver.quit()
        time.sleep(90)
        driver = webdriver.Chrome(service=service)
        driver.maximize_window()
        
driver.close()

In [17]:
image_urls_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801 entries, 0 to 0
Columns: 201 entries, 0 to 200
dtypes: object(201)
memory usage: 1.2+ MB


In [19]:
image_urls_df.reset_index(drop=True, inplace=True)

In [20]:
image_urls_df

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,Bulbasaur,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/ruby-sapphir...,https://img.pokemondb.net/sprites/diamond-pear...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/x-y/normal/b...,https://img.pokemondb.net/sprites/lets-go-pika...,https://img.pokemondb.net/sprites/sword-shield...,https://img.pokemondb.net/sprites/silver/shiny...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ivysaur,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/ruby-sapphir...,https://img.pokemondb.net/sprites/diamond-pear...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/x-y/normal/i...,https://img.pokemondb.net/sprites/lets-go-pika...,https://img.pokemondb.net/sprites/sword-shield...,https://img.pokemondb.net/sprites/silver/shiny...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Venusaur,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/ruby-sapphir...,https://img.pokemondb.net/sprites/diamond-pear...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/x-y/normal/v...,https://img.pokemondb.net/sprites/lets-go-pika...,https://img.pokemondb.net/sprites/sword-shield...,https://img.pokemondb.net/sprites/silver/shiny...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Charmander,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/ruby-sapphir...,https://img.pokemondb.net/sprites/diamond-pear...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/x-y/normal/c...,https://img.pokemondb.net/sprites/lets-go-pika...,https://img.pokemondb.net/sprites/sword-shield...,https://img.pokemondb.net/sprites/silver/shiny...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Charmeleon,https://img.pokemondb.net/sprites/red-blue/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/ruby-sapphir...,https://img.pokemondb.net/sprites/diamond-pear...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/x-y/normal/c...,https://img.pokemondb.net/sprites/lets-go-pika...,https://img.pokemondb.net/sprites/sword-shield...,https://img.pokemondb.net/sprites/silver/shiny...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,Celesteela,https://img.pokemondb.net/sprites/home/normal/...,https://img.pokemondb.net/sprites/home/shiny/c...,https://img.pokemondb.net/sprites/ultra-sun-ul...,https://img.pokemondb.net/sprites/ultra-sun-ul...,https://img.pokemondb.net/sprites/sun-moon/nor...,https://img.pokemondb.net/sprites/sun-moon/shi...,https://img.pokemondb.net/sprites/bank/normal/...,https://img.pokemondb.net/sprites/bank/shiny/c...,https://ssum-sec.casalemedia.com/usermatchredi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,Kartana,https://img.pokemondb.net/sprites/home/normal/...,https://img.pokemondb.net/sprites/home/shiny/k...,https://img.pokemondb.net/sprites/ultra-sun-ul...,https://img.pokemondb.net/sprites/ultra-sun-ul...,https://img.pokemondb.net/sprites/sun-moon/nor...,https://img.pokemondb.net/sprites/sun-moon/shi...,https://img.pokemondb.net/sprites/bank/normal/...,https://img.pokemondb.net/sprites/bank/shiny/k...,https://match.prod.bidr.io/cookie-sync/cri?r=h...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,Guzzlord,https://img.pokemondb.net/sprites/home/normal/...,https://img.pokemondb.net/sprites/home/shiny/g...,https://img.pokemondb.net/sprites/ultra-sun-ul...,https://img.pokemondb.net/sprites/ultra-sun-ul...,https://img.pokemondb.net/sprites/sun-moon/nor...,https://img.pokemondb.net/sprites/sun-mo

In [21]:
image_urls_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
count,801,792,792,792,792,792,792,788,788,783,...,1,1,1,1,1,1,1,1,1,1
unique,801,792,792,792,792,792,792,788,788,781,...,1,1,1,1,1,1,1,1,1,1
top,Togepi,https://img.pokemondb.net/sprites/sun-moon/nor...,https://img.pokemondb.net/sprites/silver/norma...,https://img.pokemondb.net/sprites/black-white/...,https://img.pokemondb.net/sprites/home/normal/...,https://img.pokemondb.net/sprites/sun-moon/nor...,https://img.pokemondb.net/sprites/home/normal/...,https://img.pokemondb.net/sprites/home/shiny/r...,https://img.pokemondb.net/sprites/silver/shiny...,https://ssum-sec.casalemedia.com/usermatchredi...,...,https://img.pokemondb.net/sprites/silver/back-...,https://img.pokemondb.net/sprites/silver/back-...,https://img.pokemondb.net/sprites/silver/back-...,https://img.pokemondb.net/sprites/silver/back-...,https://img.pokemondb.net/sprites/silver/back-...,https://img.pokemondb.net/sprites/silver/back-...,https://x.bidswitch.net/sync?ssp=adagio&user_i...,https://ad.360yield.com/server_match?partner_i...,https://ssum-sec.casalemedia.com/usermatchredi...,https://ib.adnxs.com/getuid?https%3A%2F%2Fu.4d...
freq,1,1,1,1,1,1,1,1,1,3,...,1,1,1,1,1,1,1,1,1,1


In [94]:
image_urls_df.to_csv('data/image_urls_df.csv')

## Downloading the images

In [11]:
image_urls_df = pd.read_csv('data/image_urls_df.csv')
image_urls_df = image_urls_df.drop('Unnamed: 0', axis=1)

In [47]:
# Needed to modify default urllib for the server to allow image downloads
# https://stackoverflow.com/questions/60758414/httperror-http-error-403-forbidden-or-none-is-returned-when-defining-headers-d

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.69 Safari/537.36"
urllib._urlopener = AppURLopener()

poke_url_lists = image_urls_df.values.tolist()
pokenumber = 1

# Image download had 404 error at the following pokemon (pokenumbers)
badlinks = [215, 463, 555]

for pokemon in poke_url_lists:
    
    # Skipped and manually downloaded those images
    if pokenumber in badlinks:
        pokenumber += 1
        continue
    
    imgctr = 1
    
    folder = f'{pokenumber}' + '-' + pokemon[0] + '/'
    pokemon.pop(0)
    pokenumber += 1
    
    path = "PokemonImages/" + folder
        
    for link in pokemon:
        # Create new folder for each pokemon
        if not os.path.isdir(path):
            os.mkdir(path)
            
        # Once a nan is reached,         
        if link == 'nan':
            break
        # Only save pokemon images, not ads
        elif ('pokemondb' and '.png') in str(link):
            try:
                urllib._urlopener.retrieve(link, f'{path}' + f'{imgctr}' + '.png')
                imgctr += 1
            except urllib.error.HTTPError as err:
                print(err.code)
                if err.code == 404:
                    continue   

<ipython-input-47-d44ad736ff2c>:6: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  urllib._urlopener = AppURLopener()


### Adding the generation 7 pokemon that were not in the csv

In [87]:
gen7end = ['Marshadow', 'Poipole', 'Naganadel', 'Stakataka', 'Blacephalon', 'Zeraora', 'Meltan', 'Melmetal']

driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.implicitly_wait(15)
counter = 0

gen7end_df = pd.DataFrame()
gen7end_list = []

for name in gen7end:
    urls = getimages(name, driver)
    urls = [name] + urls
    gen7end_list.append(urls)
    temp_df = pd.DataFrame(urls).T
    gen7end_df = gen7end_df.append(temp_df)
    time.sleep(15)
    
    counter +=1
    
    # In order to avoid errors from too many requests in a short time: close driver, wait, open new driver 
    if (counter % 4) == 0:
        driver.quit()
        time.sleep(90)
        driver = webdriver.Chrome(service=service)
        driver.maximize_window()
        
driver.close()

In [88]:
pokenumber = 802
for pokemon in gen7end_list:
    imgctr = 1
    
    folder = f'{pokenumber}' + '-' + pokemon[0] + '/'
    pokemon.pop(0)
    pokenumber += 1
    
    path = "PokemonImages/" + folder
        
    for link in pokemon:
        # Create new folder for each pokemon
        if not os.path.isdir(path):
            os.mkdir(path)
            
        # Once a nan is reached,         
        if link == 'nan':
            break
        # Only save pokemon images, not ads
        elif ('pokemondb' and '.png') in str(link):
            try:
                urllib._urlopener.retrieve(link, f'{path}' + f'{imgctr}' + '.png')
                imgctr += 1
            except urllib.error.HTTPError as err:
                print(err.code)
                if err.code == 404:
                    continue